# Interview Solution By Naufal Mohamed Noori

(Date: 28 December 2025)

### Problem Statement
To design and develop a data warehouse using industrial-standard framework from 3 possible datalake inputs for latter consumption by data analyst, machine learning, analystic, and technology in usable formats dictated (AWS, GCP, Azure, Snowflake, local dbt, etc.) .

Inputs:
1. CRM System - contact information, demographic information, email communications, call recordings, form submissions. 
2. Events System - registrations and attendance at student events
3. Application System - applications, education history and qualifications, statuses, certifications. 

### Final Solution
The final solution consists of gold layer data which has been transformed from the input silver layer. The solution will show aggregated result of the effectiveness for each of the methods utilized CRM, events, and applications based on the engagement made by students.

###  Proposed Architecture
I will be using medallion architecture approach to produce the final datatype consumable by various parties stated above. The medallion architecture is widely used in a lot of modern ETL tool like databricks. The proposed pipeline:

Data Source --> Data Ingestion (Landing) --> Broze Layer (Raw Data) --> Silver Later (Cleaned Data) --> Gold Layer (Transformed Data) --> Consumption (Data Ready For Users)

1. Data Source
- CRM (contacts, emails, calls, forms)
- Events (registrations, attendance)
- Applications (education history, status changes)

2. Data Ingestion
- This part is quite tricky as no 'clear' input stated from the interview documents. Hence I will make assumption that the data will come from a database and will be fetch in batch. 'Dummy' data will be created from all 3 possible sources to showcase the migration between each layer in the pipeline. As known there are bunch of sandbox opensource data related to the CRM etc out there but fro simplicity I am going to create the dummy data from scratch.

3. Bronze Layer (Raw data)
- Just an exact copy of the data pulled out from ingestion. Usually data organization are done here from unstructured and structred data. However, as simplication made in this demo we only need to deal with structure data from a simple 'demo' database

4. Silver Layer (Cleaned)
At this stage, I will perform various standard data cleanup as follows:
    - Standardized IDs
    - Clean timestamps
    - Remove duplicates
    - Handling null values (Depends if we determine null is not ok for certain column)

5. Gold Layer (Transformed)
The last stage is to create a final analytic transformed data ready for end user consumption. For this demo, I will focus on 3 outputs which are:
    - Student Dimension
        - This table will emphasizes on the student details, the origin of the students, and the program the applied for
    - Student Engagement
        - This tabe will emphasize on student interaction with QS application via emails or web appplication or any other mean of engagements (Physical signup etc). 
    - Aggregated Engagement Scores
        - This will contains some score of engagement which gauges value the students' level of connection/interest to QS

### Consumption
The final data are ready to be used by:
- BI Dashboards
- Data Science Models
- AI (If we are to handle and train ML model etc in our ETL pipeline - but in our simple case it is not - this will be the answer for the enhancement question)



In [1]:
# Import libraries

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

The next stage is to create dummy input data. The following are the snippet csv of dummy data. Possible source system ingestion including API, batch export from database, file retrieval from any cloud storage services i.e. AWS S3, Azure blob, etc

CRM Data

| student_id | email     | country | emails_sent | emails_opened | calls_logged | forms_submitted |
|-----------:|-----------|---------|------------:|--------------:|-------------:|----------------:|
| 1          | a@qs.com  | UK      | 10          | 6             | 2            | 1               |
| 2          | b@qs.com  | India   | 3           | 1             | 0            | 0               |
| 3          | c@qs.com  | USA     | 7           | 4             | 1            | 2               |

Events Data

| student_id | event_id | registered | attended | event_date           |
| ---------: | -------: | ---------: | -------: | -------------------- |
|          1 |      101 |          1 |        1 | 2025-01-05T10:00:00Z |
|          1 |      102 |          1 |        0 | 2025-01-12T14:30:00Z |
|          2 |      101 |          1 |        1 | 2025-01-08T09:00:00Z |
|          3 |      103 |          1 |        1 | 2025-01-15T16:00:00Z |
|          4 |      104 |          1 |        0 | 15-01-2025 16:00     |


Application Derived Data

| student_id | applications_started | applications_submitted | applications_accepted | highest_qualification | field_of_study   | institution_name | application_status | certifications |
| ---------: | -------------------: | ---------------------: | --------------------: | --------------------- | ---------------- | ---------------- | ------------------ | -------------- |
|          1 |                    2 |                      1 |                     1 | Bachelor's            | Engineering      | University of X  | Accepted           | IELTS          |
|          2 |                    1 |                      1 |                     0 | Bachelor's            | Business         | University of Y  | Submitted          | None           |
|          3 |                    3 |                      2 |                     1 | Master's              | Computer Science | University of Z  | Accepted           | GRE, TOEFL     |
|          4 |                    1 |                      1 |                     0 | STPM                  | Science          | SMK A            | Submitted          | STPM Science   |
|          5 |                    2 |                      2 |                     1 | SPM                   | Mathematics      | SMK B            | Accepted           | SPM Math       |
|          3 |                    3 |                      2 |                     1 | Master's              | Computer Science | University of Z  | Accepted           | GRE, TOEFL     |




In [94]:
#  Dummy Input Data Ingestion

# CRM System
crm_contacts = pd.DataFrame({
    "student_id": ["1", "2", "003", None],   # non-standard IDs, one null
    "email": ["a@qs.com", "b@qs.com", "c@qs.com", "d@qs.com"],
    "country": ["UK", "India", "USA", None], # one null
    "emails_sent": [10, 3, 7, None],         # one null
    "emails_opened": [6, 1, 4, 2],
    "calls_logged": [2, 0, 1, 0],
    "forms_submitted": [1, 0, 2, 1]
})

# Events System
events = pd.DataFrame({
    "student_id": ["1", "1", "2", "3", "3"],
    "event_id": [101, 102, 101, 103, 103], 
    "registered": [1, 1, 1, 1, 1],
    "attended": [1, 0, 1, 1, 1],
    "event_date": [
        "2025-01-05T10:00:00Z", 
        "2025-01-12T14:30:00Z", 
        "2025-01-08T09:00:00Z",
        "2025-01-15T16:00:00Z",
        "15-01-2025 16:00"  # corrupt date
    ]
})

# Application System
applications = pd.DataFrame({
    "student_id": [1, 2, 3, 4, 5, 3],  # duplicate student_id = 3
    "applications_started": [2, 1, 3, 1, 2, 3],
    "applications_submitted": [1, 1, 2, 1, 2, 2],
    "applications_accepted": [1, 0, 1, 0, 1, 1],
    "highest_qualification": [
        "Bachelor's",
        "Bachelor's",
        "Master's",
        "STPM",
        "SPM",
        "Master's"
    ],
    "field_of_study": [
        "Engineering",
        "Business",
        "Computer Science",
        "Science",
        "Mathematics",
        "Computer Science"
    ],
    "institution_name": [
        "University of X",
        "University of Y",
        "University of Z",
        "SMK A",
        "SMK B",
        "University of Z"
    ],
    "application_status": [
        "Accepted",
        "Submitted",
        "Accepted",
        "Submitted",
        "Accepted",
        "Accepted"
    ],
    "certifications": [
        "IELTS",
        None,
        "GRE, TOEFL",
        "STPM Science",
        "SPM Math",
        "GRE, TOEFL"
    ]
})


print("=========================================================================================================")
print(f"THIS IS THE BRONZE DATA LAYER AFTER ALL RAW DATA ARE INGESTION")
print("=========================================================================================================\n")

print(crm_contacts.head())
print(events.head())
print(applications.head())

THIS IS THE BRONZE DATA LAYER AFTER ALL RAW DATA ARE INGESTION

  student_id     email country  emails_sent  emails_opened  calls_logged  \
0          1  a@qs.com      UK         10.0              6             2   
1          2  b@qs.com   India          3.0              1             0   
2        003  c@qs.com     USA          7.0              4             1   
3       None  d@qs.com    None          NaN              2             0   

   forms_submitted  
0                1  
1                0  
2                2  
3                1  
  student_id  event_id  registered  attended            event_date
0          1       101           1         1  2025-01-05T10:00:00Z
1          1       102           1         0  2025-01-12T14:30:00Z
2          2       101           1         1  2025-01-08T09:00:00Z
3          3       103           1         1  2025-01-15T16:00:00Z
4          3       103           1         1      15-01-2025 16:00
   student_id  applications_started  application

The next stage in this pipeline is to create a silver data  - to cleanup data resides within bronze container/partition/catalogue. The first stage is to run any functions needed to perform data cleanup. Function is important to create modular and standardize process accross the ETL. In this example I will create a function to handle date conversion to ISO Z format

In [70]:

def check_convert_date_to_iso_z(value):
    """
    Convert any value to ISO 8601 UTC (Z) format.
    Returns pd.Timestamp or pd.NaT
    """
    if pd.isna(value):
        return pd.NaT

    # If already a Timestamp, ensure UTC
    if isinstance(value, pd.Timestamp):
        if value.tzinfo is None:
            return value.tz_localize('UTC')
        else:
            return value.tz_convert('UTC')

    val_str = str(value).strip()

    # Try pandas flexible parser with dayfirst=True
    try:
        ts = pd.to_datetime(val_str, errors='coerce', dayfirst=True)
        if pd.notna(ts):
            if ts.tzinfo is None:
                ts = ts.tz_localize('UTC')
            else:
                ts = ts.tz_convert('UTC')
            return ts
    except:
        pass

    # Try a few common formats manually
    from datetime import datetime
    import pytz
    formats = ["%d-%m-%Y %H:%M", "%d/%m/%Y %H:%M", "%Y/%m/%d %H:%M", "%Y-%m-%d %H:%M"]
    for fmt in formats:
        try:
            dt = datetime.strptime(val_str, fmt)
            return pd.Timestamp(dt.replace(tzinfo=pytz.UTC))
        except:
            continue

    # Could not parse
    return pd.NaT


In [39]:
# Silver layer CRM

# Standardize IDs by converting to string, remove leading zeros
crm_silver = crm_contacts.copy()
crm_silver['student_id'] = crm_silver['student_id'].astype(str).str.lstrip('0')

# Handle nulls: drop rows where student_id is missing (key column)
crm_silver = crm_silver.dropna(subset=['student_id'])

# Fill missing numeric values with 0 (for emails_sent)
crm_silver['emails_sent'] = crm_silver['emails_sent'].fillna(0)

# Fill missing categorical values with standard placeholder like unknown or null with Unknown
crm_silver['country'] = crm_silver['country'].fillna('Unknown')

# Remove duplicates
crm_silver = crm_silver.drop_duplicates(subset=['student_id'])

crm_silver.reset_index(drop=True, inplace=True) #This is just to sort the rows with meaningful one on top
crm_silver

,student_id,email,country,emails_sent,emails_opened,calls_logged,forms_submitted
0,1,a@qs.com,UK,10.0,6,2,1
1,2,b@qs.com,India,3.0,1,0,0
2,3,c@qs.com,USA,7.0,4,1,2
3,None,d@qs.com,Unknown,0.0,2,0,1


In [72]:
# Silver Layer Events

events_silver = events.copy()

# Standardize IDs
events_silver['student_id'] = events_silver['student_id'].astype(str).str.lstrip('0')

# Clean timestamps
events_silver['event_date_clean'] = pd.to_datetime(
    events_silver['event_date'], errors='coerce', utc=True
)

# Fix date format using date conversion function
events['event_date_clean'] = events['event_date'].apply(check_convert_date_to_iso_z)

# Remove duplicates based on student_id + event_id + event_date_clean
events_silver = events_silver.drop_duplicates(subset=['student_id','event_id','event_date_clean'])

events_silver.reset_index(drop=True, inplace=True)
events_silver


C:\Users\naufa\AppData\Local\Temp\ipykernel_14752\1094773874.py:20: UserWarning: Parsing dates in %Y-%m-%dT%H:%M:%S%z format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  ts = pd.to_datetime(val_str, errors='coerce', dayfirst=True)


,student_id,event_id,registered,attended,event_date,event_date_clean
0,1,101,1,1,2025-01-05T10:00:00Z,2025-01-05 10:00:00+00:00
1,1,102,1,0,2025-01-12T14:30:00Z,2025-01-12 14:30:00+00:00
2,2,101,1,1,2025-01-08T09:00:00Z,2025-01-08 09:00:00+00:00
3,3,103,1,1,2025-01-15T16:00:00Z,2025-01-15 16:00:00+00:00
4,3,103,1,1,15-01-2025 16:00,NaT


In [75]:
# Silver Layer Application Derived Data

applications_silver = applications.copy()

# Standardize IDs
applications_silver['student_id'] = applications_silver['student_id'].astype(str).str.lstrip('0')

# Handle nulls with standard None for string and  for number type col
applications_silver['applications_started'] = applications_silver['applications_started'].fillna(0)
applications_silver['highest_qualification'] = applications_silver['highest_qualification'].fillna('None')
applications_silver['institution_name'] = applications_silver['institution_name'].fillna('None')
applications_silver['certifications'] = applications_silver['certifications'].fillna('None')

# Remove duplicates
applications_silver = applications_silver.drop_duplicates(subset=['student_id'])

applications_silver.reset_index(drop=True, inplace=True)
applications_silver

,student_id,applications_started,applications_submitted,applications_accepted,highest_qualification,field_of_study,institution_name,application_status,certifications
0,1,2,1,1,Bachelor's,Engineering,University of X,Accepted,IELTS
1,2,1,1,0,Bachelor's,Business,University of Y,Submitted,None
2,3,3,2,1,Master's,Computer Science,University of Z,Accepted,"GRE, TOEFL"
3,4,1,1,0,STPM,Science,SMK A,Submitted,STPM Science
4,5,2,2,1,SPM,Mathematics,SMK B,Accepted,SPM Math


The last stage is to transform to Gold Layer involves the combination and aggreation of silve layers data to create a meaningful data for later consumption. The gold layer requires business requirement - how the data needs to be used at the end. In our sample, the main question is to identify:

**_WHICH STUDENTS ARE HIGHLY ENGAGED, WHY AND HOW???_**

As discussed at the beginnining, I will provide 3 possible schema/golden data layer for current framework:

1. Student dimension (Summarize version of schema based on combination of all 3 different inputs). This will be useful for any Power BI tool consumption
2. Student engagement (Showcase the student enagement based on 3 inputs) - again useful for Power BI consumption
3. Aggregated engagement score (Score the student based on the engagement and return the rank). Scoring is based on the following criteria:
    - email engagement score times the weighting
    - event engagement score time the weighting
    - application engagement score times the weighting

    ***Weighting is the proportion of how important of each criteria given to the evaluation. This can be adjusted based on the business input and engagement (Possible from data analytic too - measuring the cost (by input) per registration etc)

    The final aggregated engagement score is important for backend application i.e. follow up email, business development calls, or business decisison to gauge effectiveness of any campaign made.

In [82]:
# Email respons WEIGHTS
STARTED = 10
SUBMITTED = 20
ACCEPTED = 30

# Engagement score WEIGHTS
EMAIL_ENGAGEMENT_WEIGHT = 0.3
EVENT_ENGAGEMENT_WEIGHT = 0.3
APPLICATION_ENGAGEMENT_WEIGHT = 0.4

In [77]:
# Dimension Layer

dim_student = (
    crm_silver[["student_id", "email", "country"]]
    .merge(
        applications_silver[
            ["student_id", "highest_qualification", "field_of_study", "institution_name", "certifications"]
        ],
        on="student_id",
        how="left"
    )
    .drop_duplicates("student_id")
)

dim_student

,student_id,email,country,highest_qualification,field_of_study,institution_name,certifications
0,1,a@qs.com,UK,Bachelor's,Engineering,University of X,IELTS
1,2,b@qs.com,India,Bachelor's,Business,University of Y,None
2,3,c@qs.com,USA,Master's,Computer Science,University of Z,"GRE, TOEFL"
3,None,d@qs.com,Unknown,NaN,NaN,NaN,NaN


In [ ]:
# Student Engagement

# Email engagement
crm_silver["email_engagement_score"] = np.where(
    crm_silver["emails_sent"] > 0,
    (crm_silver["emails_opened"] / crm_silver["emails_sent"]) * 100,
    0
)

# Event engagement (Extract statistical view)
event_agg = events_silver.groupby("student_id").agg(
    events_registered=("event_id", "count"),
    events_attended=("attended", "sum"),
    last_event_at=("event_date", "max")
).reset_index()

event_agg["event_engagement_score"] = (
    event_agg["events_attended"] /
    event_agg["events_registered"]
).fillna(0) * 100

# Application engagement
applications_silver["application_engagement_score"] = (
    applications_silver["applications_started"] * STARTED +
    applications_silver["applications_submitted"] * SUBMITTED +
    applications_silver["applications_accepted"] * ACCEPTED
)

print("=========================================================================================================")
print(f"THIS IS THE GOLD DATA LAYER")
print("=========================================================================================================\n")

print(crm_silver.loc[:,['student_id','email_engagement_score']].head())
print(event_agg.loc[:,['student_id','event_engagement_score']].head())
print(applications_silver.loc[:,['student_id','application_engagement_score']].head())


THIS IS THE GOLD DATA LAYER

  student_id  email_engagement_score
0          1               60.000000
1          2               33.333333
2          3               57.142857
3       None                     inf
  student_id  event_engagement_score
0          1                    50.0
1          2                   100.0
2          3                   100.0
  student_id  application_engagement_score
0          1                            70
1          2                            30
2          3                           100
3          4                            30
4          5                            90


In [95]:
# Final Student Engagement Table

gold_student_engagement = (
    crm_silver[["student_id", "email_engagement_score"]]
    .merge(
        event_agg[["student_id", "event_engagement_score", "last_event_at"]],
        on="student_id",
        how="left"
    )
    .merge(
        applications_silver[["student_id", "application_engagement_score"]],
        on="student_id",
        how="left"
    )
    .fillna(0)
)

gold_student_engagement["total_engagement_score"] = (
    gold_student_engagement["email_engagement_score"] * EMAIL_ENGAGEMENT_WEIGHT +
    gold_student_engagement["event_engagement_score"] * EVENT_ENGAGEMENT_WEIGHT +
    gold_student_engagement["application_engagement_score"] * APPLICATION_ENGAGEMENT_WEIGHT
)


# Statiscal labelling for each students engagement result
gold_student_engagement["engagement_level"] = pd.cut(
    gold_student_engagement["total_engagement_score"],
    bins=[-1, 40, 75, 1000],
    labels=["Low", "Medium", "High"]
)

gold_student_engagement

,student_id,email_engagement_score,event_engagement_score,last_event_at,application_engagement_score,total_engagement_score,engagement_level
0,1,60.000000,50.0,2025-01-12T14:30:00Z,70.0,61.000000,Medium
1,2,33.333333,100.0,2025-01-08T09:00:00Z,30.0,52.000000,Medium
2,3,57.142857,100.0,2025-01-15T16:00:00Z,100.0,87.142857,High
3,None,inf,0.0,0,0.0,inf,NaN


Additional Questions:

•	Describe how you would handle ingestion and orchestration

    - Handling ingestion using probably databricks bulit in workflow. Using pyspark to orchestrate big data.

•	Describe how this data will be used downstream and how it helps the business achieve the objective.

    - As mentioned earlier, data con be consumed by tool olike Power BI or business manager to approach prospective student or even any scheduler/cron backend application to send out email/whatsapp follow up to prospective students.

•	[Optional] If you are comfortable with graph thinking, sketch how this would map to node-edge form. 

    - Simple node drawn in first markdown page

•	[Optional] Include tools and services involved in the implementation of this model. 

    - Databricks, pandas, pyspark

•	[Optional] Include other aspects of the solution that you believe would be critical to its success, e.g. data lineage, access controls, data delivery mechanisms, AI enablement, real-time data streaming.

    - We can also use a json schema with versioninig, acl, entitlement groups so backend application will consume the records and control all the relevant functionality and security accordingly. One of the example I am currently utilize is here: 

    [Link Here](https://community.opengroup.org/osdu/data/data-definitions)


•	[Optional] Propose further enhancements in Phase 2.

    - Include machine learning model training within the pipeline. Use viable machine learning algorith from Keras to refine and train ML model for end user consumption. At the end, we can predict prospective students based on criteria given and focusing on them starts from the application submission until results.
